问题主要分为两个大类，一部分需要使用SQL从金融数据库中提取相关信息；另一部分则需要从公司的招股说明书中提取有效信息。

这里采用`Tongyi-Finance-14B-Chat`大模型对于所有问题先进行一个合理的分类

In [1]:
# 导入问题
import jsonlines

def read_jsonl(path):
    content = []
    with jsonlines.open(path, "r") as json_file:
        for obj in json_file.iter(type=dict, skip_invalid=True):
            content.append(obj)
    return content

questions = read_jsonl('bs_challenge_financial_14b_dataset/question.json')

In [2]:
# 导入模型
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

# model_dir = snapshot_download('TongyiFinance/Tongyi-Finance-14B-Chat', local_dir='/root/autodl-tmp/models/Tongyi-Finance-14B-Chat')
# model_dir = snapshot_download('TongyiFinance/Tongyi-Finance-14B-Chat', local_dir='/root/autodl-tmp/models/Tongyi-Finance-14B-Chat')
model_dir = '/root/autodl-tmp/models/Tongyi-Finance-14B-Chat'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map='cuda', trust_remote_code=True, bf16=True).eval()
model.generation_config = GenerationConfig.from_pretrained(model_dir,
                                                           trust_remote_code=True,
                                                           temperature = 0.0000001,
                                                           top_p = 1,
                                                           do_sample = False,
                                                           seed = 1234)


# 构建prompt
prompt = """
    你是一个问题分类器。对于每个提供给你的问题，你需要猜测答案是在该公司的招股说明书中还是在基金股票数据库里。以下是一些例子：

    问题：“在2019年的中期报告里，XX基金管理有限公司管理的基金中，有多少比例的基金是个人投资者持有的份额超过机构投资者？希望得到一个精确到两位小数的百分比。”
    回答：“基金股票数据库”
    
    问题：“XXXX股份有限公司变更设立时作为发起人的法人有哪些？”
    回答：“该公司的招股说明书”
    
    问题：“我想知道XXXXXX债券A基金在20200930的季报中，其可转债持仓占比最大的是哪个行业？用申万一级行业来统计。”
    回答：“基金股票数据库”
    
    问题：“XXXXXX股份有限公司2020年增资后的投后估值是多少？”
    回答：“该公司的招股说明书”
    
    问题：“根据XXXXXX股份有限公司招股意向书，全球率先整体用LED路灯替换传统路灯的案例是？”
    回答：“该公司的招股说明书”
    
    问题：“什么公司、在何时与XXXXXX股份有限公司发生了产品争议事项？产品争议事项是否已经解决？”
    回答：“该公司的招股说明书”
    
    问题：“请帮我查询下股票代码为XXXXXX的股票在2021年内最高日收盘价是多少？”
    回答：“基金股票数据库”
    
    问题：“XXXXXX股份有限公司的中标里程覆盖率为多少？”
    回答：“该公司的招股说明书”
    
    问题：“根据中国证监会颁布的《上市公司行业分类指导》的规定，XXXXXX有限公司所属的行业大类、中类、小类是什么？”
    回答：“该公司的招股说明书”
    
    问题：“请问XXXX年一季度有多少家基金是净申购?它们的净申购份额加起来是多少?请四舍五入保留小数点两位。”
    回答：“基金股票数据库”
    
    问题：“XXXXXX有限公司和合肥翰林是否按规定为员工缴纳了社会保险？”
    回答：“该公司的招股说明书”
    
    问题：“我想知道XXXXXX有限公司在2020年成立了多少只管理费率小于0.8%的基金？”
    回答：“基金股票数据库”
    
    问题：“根据《CRCC产品认证实施规则》，《铁路产品认证证书》有效期为多久？XXXXXX有限公司取得 《铁路产品认证证书》后，至少多久需要接受一次监督？”
    回答：“该公司的招股说明书”
    
    问题：“我想知道XXXXXX基金管理有限公司在2019年成立了多少只管理费率小于0.8%的基金？”
    回答：“基金股票数据库”
    
    问题：“请问XXXX年一季度有多少家基金是净申购?它们的净申购份额加起来是多少?请四舍五入保留小数点两位。”
    回答：“基金股票数据库”
    
    问题：“我想知道XXXXXX有限公司在2019年成立了多少只管理费率小于0.8%的基金？”
    回答：“基金股票数据库”
    
    问题：“我想知道股票XXXXXX在申万行业分类下的二级行业是什么？用最新的数据。”
    回答：“基金股票数据库”
    
    问题：“请帮我查询下股票代码为XXXXXX的股票在2019年内最高日收盘价是多少？”
    回答：“基金股票数据库”
    
    问题：“股票XXXXXX在20200227日期中的收盘价是多少?（小数点保留3位）”
    回答：“基金股票数据库”
    
    问题：“截至2009年底，中海达、南方测绘合计占有国产品牌销售额的多大比例？”
    回答：“该公司的招股说明书”
    
    问题：“截止2005年12月31日，南岭化工厂的总资产和净资产分别是多少？”
    回答：“该公司的招股说明书”
    
    问题：“股票XXXXXX在20200227日期中的收盘价是多少?（小数点保留3位）”
    回答：“基金股票数据库”

    根据上面提供的例子对以下问题进行分类。
    问题：“
    """

In [ ]:
# 对每个问题进行分类
for q in questions:
    tmp_question = q['question']
    tmp_prompt = prompt + tmp_question + "?”"
    
    response, history = model.chat(tokenizer, tmp_prompt, history=None)
    
    if '招股说明书' in response and '股票数据库' not in response:
        tmp_class = 'text'
    elif '招股说明书' not in response and '股票数据库' in response:
        tmp_class = 'sql'
    else:
        tmp_class = 'sql'
    
    q['question_type'] = tmp_class

questions[0]    

## 将分类后的问题保存下来

In [ ]:
def write_jsonl(path, content):
    with jsonlines.open(path, "w") as json_file:
        json_file.write_all(content)
        
write_jsonl('bs_challenge_financial_14b_dataset/classified_question.json', questions)